In [1]:
import nibabel as nib       #read/write access to some common neuroimaging fomrates like NIFTI1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom as dicom
import random
import itk      # Jupyter widgets to visualize images in 2D and 3D
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons 


Read and load images

In [2]:
#loading nifti1 image 
image_nii=nib.load('../ass2/data/sub-62038_ses-1_acq-t1csmp2ragesag06mmUNIDEN_T1w.nii')

#loading a dicom slice of different dataset
image_dicom=dicom.dcmread('../ass2/dcm_data/image-00100.dcm')



reading metadata from dicom image

In [3]:
for el in image_dicom:
    print(f"{el.tag} - {el.name} : {el.value}")
#image_dicom=image_dicom.pixel_array

(0008,0005) - Specific Character Set : ISO_IR 100
(0008,0008) - Image Type : ['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']
(0008,0012) - Instance Creation Date : 20061012
(0008,0013) - Instance Creation Time : 091632.000000
(0008,0016) - SOP Class UID : 1.2.840.10008.5.1.4.1.1.2
(0008,0018) - SOP Instance UID : 1.2.826.0.1.3680043.8.1055.1.20111102150817669.41237978.47338332
(0008,0020) - Study Date : 20061012
(0008,0022) - Acquisition Date : 20061012
(0008,0023) - Content Date : 20061012
(0008,0030) - Study Time : 090258.000000
(0008,0032) - Acquisition Time : 085229.000000
(0008,0033) - Content Time : 085232.844002
(0008,0060) - Modality : CT
(0008,1030) - Study Description : CT1 abdomen
(0008,1032) - Procedure Code Sequence : [(0008,0100) Code Value                          SH: 'CTABDOM'
(0008,0102) Coding Scheme Designator            SH: 'XPLORE'
(0008,0104) Code Meaning                        LO: 'CT1 abdomen']
(0008,103E) - Series Description : ARTERIELLE
(0008,1111) - Referenced Per

Invalid value for VR UI: '1.2.826.0.1.3680043.8.1055.1.20111102150758591.96842950.07877442'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.


internal structure of NIFTI1

In [4]:
header=image_nii.header
print(header)
print()

#gets the shape of the image
height, width, depth= image_nii.get_fdata().shape
print(f"height: {height}, width: {width}, depth: {depth}" )
print()

#affine matrix
print(f"affine transformation matrix:")
affine=image_nii.affine
print(image_nii.affine)
print()

#voxel spacing (x_spacing, y_spacing, z_spacing)
voxel_spacing=header.get_zooms()
print(f"voxel spacing: ",*voxel_spacing)


<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : np.bytes_(b'')
db_name         : np.bytes_(b'')
extents         : 0
session_error   : 0
regular         : np.bytes_(b'r')
dim_info        : 54
dim             : [  3 256 362 384   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.    0.63  0.625 0.625 6.    0.    0.    0.   ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : np.bytes_(b'TE=2.1;Time=132725.145;phase=1')
aux_file        : np.bytes_(b'Denoised Image (lambda ')
qform_code      : scanner
sform_code      : scanner
quatern_b       : -0.047692526
quatern_c       : -0.02

stacking dicom images

In [5]:
import os

def stacking(directory_path):
    dicom_files=[dicom.dcmread(os.path.join(directory_path,f)) 
                 for f in os.listdir(directory_path)]
    
    #to decide along what direction the slices have been made
    def normal_to_slice(ds):
        iop=[float(v) for v in ds.ImageOrientationPatient]
        row_dir=np.array(iop[0:3])
        col_dir=np.array(iop[3:6])
        normal=np.cross(row_dir, col_dir)
        return normal
    positions = [np.dot(normal_to_slice(ds),ds.ImagePositionPatient) for ds in dicom_files]
    
    #sorting the files based on the dot product of the top-most left point coordinates and the vector perpendicular to the plane of the image
    dicom_files.sort(key=lambda x:positions)

    #stacking the images
    image_stack=np.stack([ds.pixel_array for ds in dicom_files], axis=0)
    return image_stack

directory='../ass2/dcm_data'
volume_3d=stacking(directory)

print("3D array's shape: ", volume_3d.shape)

3D array's shape:  (361, 512, 512)


Visualization of slices in different planes

In [6]:
import scipy.ndimage as ndi
def axial_layer(layer):
    plt.figure(figsize=(10,5))
    plt.title(f"Axial layer {layer}")
    plt.imshow(ndi.rotate(image_nii.get_fdata()[:,:,layer],90), cmap='bone')
    #plt.imshow(image_nii.get_fdata()[:,:,layer],cmap='bone', origin='lower')
    plt.axis('off')
    return layer
interact(axial_layer, layer=(0,depth-1));

def saggital_layer(layer2):
    plt.figure(figsize=(10,5))
    plt.title(f"Saggital layer {layer2}")
    plt.imshow(ndi.rotate(image_nii.get_fdata()[layer2,:,:],90), cmap='bone')
    plt.axis('off')
    return layer2
interact(saggital_layer, layer2=(0,height-1));

def coronal_layer(layer3):
    plt.figure(figsize=(10,5))
    plt.title(f"Coronal layer {layer3}")
    plt.imshow(ndi.rotate(image_nii.get_fdata()[:,layer3,:],90), cmap='bone')
    plt.axis('off')
    return layer3
interact(coronal_layer, layer3=(0,width-1));

interactive(children=(IntSlider(value=191, description='layer', max=383), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=127, description='layer2', max=255), Output()), _dom_classes=('widget-in…

interactive(children=(IntSlider(value=180, description='layer3', max=361), Output()), _dom_classes=('widget-in…

image orientation for nifti

In [10]:
from nibabel.orientations import aff2axcodes

#printing orientation codes
affine=image_nii.affine
print(aff2axcodes(affine))

#voxel size
voxel_size=np.linalg.norm(affine[:3,:3],axis=0)
print("Voxel sizes (x,y,z): ",voxel_size)

#unit vectors along image axes (each column of affine matrix is unit drection vector for corresponding image axis)
directions = affine[:3,:3]/voxel_size
print("Direction vectors:")
print(directions)

#image origin
print("Image origin:", affine[:3,3])



('R', 'A', 'S')
Voxel sizes (x,y,z):  [0.62999954 0.62500003 0.62500001]
Direction vectors:
[[ 0.99853114 -0.02129155 -0.04982194]
 [ 0.02594323  0.99517111  0.09466469]
 [ 0.0475658  -0.09581818  0.99426172]]
Image origin: [ -73.60540771 -100.073349   -119.9901886 ]


orientation handling in dicom 

In [12]:
#plt.imshow(image_dicom, cmap='gray')
#ImageOrientationPatient gives the unti vectors for row direction and column direction
#[Rx,Ry,Rz,Cx,Cy,Cz], these are in patient-based coordinates, ie, X: Left to Right, Y: Posterior to Anterior, Z: Inferior to Superior
#so[1,0,0,0,1,0] means, row vector [1,0,0] -> direction along axis 1(column) -> along X (Left to Right) axis

#imagepositionpatient gives the real-world coordinates of the upper-left most corner of the image 

ds=image_dicom
orientation=ds.ImageOrientationPatient
position= ds.ImagePositionPatient        
pixel_spacing = ds.PixelSpacing       
row_dir=np.array(orientation[0:3])
col_dir=np.array(orientation[3:6])
normal=np.cross(row_dir,col_dir)
spacing=np.array([pixel_spacing[1],pixel_spacing[0],1.0])
or_matrix=np.stack([row_dir, col_dir, normal], axis=1) * spacing
affine = np.eye(4)
affine[:3,:3]=or_matrix
affine[:3,3]=position
print("Affine matrix:")
print(affine)


Affine matrix:
[[ 5.89843750e-01  0.00000000e+00  0.00000000e+00 -1.51493508e+02]
 [ 0.00000000e+00  5.89843750e-01  0.00000000e+00 -3.66564417e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.24500000e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
